# Hierarchical Bayesian Logistic Regression using PyStan
**Florian Ott, 2021**

Here we fit the three models: Planning (PM), Simple (SM) and Hybrid (HM). Further explanation of the models can be found in the main manuscript.

In [2]:
import numpy as np
import pandas as pd
import pystan
import arviz as az
import glob as glob
import time as time

# Load particpant data 
filename = glob.glob('../data/behaviour/data_all_partipants_20210623102746.csv') 
dat = pd.read_csv(filename[0],index_col = 0)

## Specifying the models 

In [3]:
# Stan code for the PM and SM 
standard_m1 = '''
data {
  int<lower=0> N;
  int<lower=0,upper=1> response[N];
  vector[N] dv;
  vector[N] is_basic;
  vector[N] is_full_energy;
  vector[N] is_low_energy_LC;
  vector[N] is_low_energy_HC;
  int<lower=0> N_subjects;
  int<lower = 1> vpn[N];  
}

parameters {
//hyper parameters
  real mu_theta_basic;
  real mu_beta_dv;
  real<lower=0> sigma_theta_basic;
  real<lower=0> sigma_beta_dv;
  
//parameters 
  vector[N_subjects] theta_basic;
  real theta_full_energy;
  real theta_low_energy_LC;
  real theta_low_energy_HC;
  vector[N_subjects] beta_dv;
}

model {
//hyper priors
  mu_theta_basic ~ normal(0,2);
  mu_beta_dv ~ normal(0,2);
  sigma_theta_basic ~ normal(0,2);
  sigma_beta_dv ~ normal(0,2);

// priors 
  theta_basic ~ normal(mu_theta_basic, sigma_theta_basic);
  theta_full_energy ~ normal(0, 2);
  theta_low_energy_LC ~ normal(0, 2);
  theta_low_energy_HC ~ normal(0, 2);
  beta_dv ~ normal(mu_beta_dv,sigma_beta_dv);  

// likelihood 
  response ~ bernoulli_logit(theta_full_energy * is_full_energy + theta_low_energy_LC * is_low_energy_LC + theta_low_energy_HC * is_low_energy_HC + theta_basic[vpn] .* is_basic + beta_dv[vpn] .* dv);
}

generated quantities {
  vector[N] log_lik;
  vector[N] response_new;
  vector[N_subjects] theta_basic_rep;
  vector[N_subjects] beta_dv_rep;

// pointwise log-likelihood
  for (n in 1:N) {
    log_lik[n] = bernoulli_logit_lpmf(response[n]  |  (theta_full_energy * is_full_energy[n] + theta_low_energy_LC * is_low_energy_LC[n] + theta_low_energy_HC * is_low_energy_HC[n] + theta_basic[vpn[n]] * is_basic[n] + beta_dv[vpn[n]] * dv[n]));
    }

// posterior predictive simulation  
  for (n in 1:N_subjects){
    theta_basic_rep[n] = normal_rng(mu_theta_basic, sigma_theta_basic);
    beta_dv_rep[n] = normal_rng(mu_beta_dv, sigma_beta_dv);
    }

  for (n in 1:N){
    response_new[n] = bernoulli_logit_rng(theta_full_energy * is_full_energy[n] + theta_low_energy_LC * is_low_energy_LC[n] + theta_low_energy_HC * is_low_energy_HC[n] + theta_basic_rep[vpn[n]] * is_basic[n] + beta_dv_rep[vpn[n]] * dv[n]);
    } 
}
'''

In [4]:
# Stan code for the HM model.
# This includes four offer specific biases. 
standard_m2 = '''
data {
  int<lower=0> N;
  int<lower=0,upper=1> response[N];
  vector[N] dv;
  vector[N] is_basic_1;
  vector[N] is_basic_2;
  vector[N] is_basic_3;
  vector[N] is_basic_4;
  vector[N] is_full_energy;
  vector[N] is_low_energy_LC;
  vector[N] is_low_energy_HC;
  int<lower=0> N_subjects;
  int<lower = 1> vpn[N];  
}

parameters {
// hyper paramters 
  real mu_theta_basic_1;
  real mu_theta_basic_2;
  real mu_theta_basic_3;
  real mu_theta_basic_4;
  real mu_beta_dv;  
  real<lower=0> sigma_theta_basic_1;
  real<lower=0> sigma_theta_basic_2;
  real<lower=0> sigma_theta_basic_3;
  real<lower=0> sigma_theta_basic_4;
  real<lower=0> sigma_beta_dv;

// parameters
  vector[N_subjects] theta_basic_1;
  vector[N_subjects] theta_basic_2;
  vector[N_subjects] theta_basic_3;
  vector[N_subjects] theta_basic_4;
  real theta_full_energy;
  real theta_low_energy_LC;
  real theta_low_energy_HC;
  vector[N_subjects] beta_dv;
}

model {
//hyper priors
  mu_theta_basic_1 ~ normal(0,2);
  mu_theta_basic_2 ~ normal(0,2);
  mu_theta_basic_3 ~ normal(0,2);
  mu_theta_basic_4 ~ normal(0,2);
  mu_beta_dv ~ normal(0,2);  
  sigma_theta_basic_1 ~ normal(0,2);
  sigma_theta_basic_2 ~ normal(0,2);
  sigma_theta_basic_3 ~ normal(0,2);
  sigma_theta_basic_4 ~ normal(0,2);
  sigma_beta_dv ~ normal(0,2);

// priors
  theta_basic_1 ~ normal(mu_theta_basic_1,sigma_theta_basic_1);
  theta_basic_2 ~ normal(mu_theta_basic_2,sigma_theta_basic_2);
  theta_basic_3 ~ normal(mu_theta_basic_3,sigma_theta_basic_3);
  theta_basic_4 ~ normal(mu_theta_basic_4,sigma_theta_basic_4);
  theta_full_energy ~ normal(0,2);
  theta_low_energy_LC ~ normal(0,2);
  theta_low_energy_HC ~ normal(0,2);
  beta_dv ~ normal(mu_beta_dv,sigma_beta_dv);  

// likelihood 
  response ~ bernoulli_logit(theta_full_energy * is_full_energy + theta_low_energy_LC * is_low_energy_LC + theta_low_energy_HC * is_low_energy_HC + theta_basic_1[vpn] .* is_basic_1 + theta_basic_2[vpn] .* is_basic_2 + theta_basic_3[vpn] .* is_basic_3 + theta_basic_4[vpn] .* is_basic_4 + beta_dv[vpn] .* dv);
}

generated quantities {
  vector[N] log_lik;
  vector[N] response_new;
  vector[N_subjects] theta_basic_1_rep;
  vector[N_subjects] theta_basic_2_rep;
  vector[N_subjects] theta_basic_3_rep;
  vector[N_subjects] theta_basic_4_rep;
  vector[N_subjects] beta_dv_rep;

// pointwise log-likelihood
  for (n in 1:N) {
    log_lik[n] = bernoulli_logit_lpmf(response[n]  |  (theta_full_energy * is_full_energy[n] + theta_low_energy_LC * is_low_energy_LC[n] + theta_low_energy_HC * is_low_energy_HC[n] + theta_basic_1[vpn[n]] * is_basic_1[n] + theta_basic_2[vpn[n]] * is_basic_2[n] + theta_basic_3[vpn[n]] * is_basic_3[n] + theta_basic_4[vpn[n]] * is_basic_4[n] + beta_dv[vpn[n]] * dv[n]));
    }

// posterior predictive simulation  
  for (n in 1:N_subjects){
    theta_basic_1_rep[n] = normal_rng(mu_theta_basic_1, sigma_theta_basic_1);
    theta_basic_2_rep[n] = normal_rng(mu_theta_basic_2, sigma_theta_basic_2);
    theta_basic_3_rep[n] = normal_rng(mu_theta_basic_3, sigma_theta_basic_3);
    theta_basic_4_rep[n] = normal_rng(mu_theta_basic_4, sigma_theta_basic_4);
    beta_dv_rep[n] = normal_rng(mu_beta_dv, sigma_beta_dv);
    }  

  for (n in 1:N){
    response_new[n] = bernoulli_logit_rng(theta_full_energy * is_full_energy[n] + theta_low_energy_LC * is_low_energy_LC[n] + theta_low_energy_HC * is_low_energy_HC[n] + theta_basic_1_rep[vpn[n]] * is_basic_1[n] + theta_basic_2_rep[vpn[n]] * is_basic_2[n] + theta_basic_3_rep[vpn[n]] * is_basic_3[n] + theta_basic_4_rep[vpn[n]] * is_basic_4[n] + beta_dv_rep[vpn[n]] * dv[n]);
    } 
}
'''

## Compiling

In [5]:
sm_standard_m1 = pystan.StanModel(model_code=standard_m1,verbose = False)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_430bf6b44e78bc5e7461815b80fab04b NOW.


In [6]:
sm_standard_m2 = pystan.StanModel(model_code=standard_m2,verbose = False)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_537ce8db94eafff0262fe8a4e5d5afbb NOW.


## Specifying the data

### Simple

In [7]:
n_iter = 2000
n_warmup = 1000
n_chains = 4
param_names_simple = ['theta_full_energy', 'theta_low_energy_LC','theta_low_energy_HC', 'theta_basic','beta_dv']

idx = (dat['timeout'] == 0)
response = (dat.loc[idx,['response']] == 0).to_numpy(dtype='int').squeeze()
is_full_energy = dat.loc[idx,['is_full_energy']].to_numpy(dtype='int').squeeze()
is_low_energy_LC = dat.loc[idx,['is_low_energy_LC']].to_numpy(dtype='int').squeeze()
is_low_energy_HC = dat.loc[idx,['is_low_energy_HC']].to_numpy(dtype='int').squeeze()
is_basic = dat.loc[idx,['is_basic']].to_numpy(dtype='int').squeeze()
dv = dat.loc[idx,['dv_simple']].to_numpy().squeeze()
vpn = dat.loc[idx,['vpn']].to_numpy().squeeze() - 100
N_subjects = len(np.unique(vpn))

dat_dict_simple = {'N':len(response),         
            'response':response,
            'dv':dv, 
            'is_full_energy':is_full_energy ,
            'is_low_energy_LC':is_low_energy_LC,
            'is_low_energy_HC':is_low_energy_HC,
            'is_basic':is_basic,
            'N_subjects':N_subjects,
            'vpn':vpn
            } 

### Planning

In [8]:
n_iter = 2000
n_warmup = 1000
n_chains = 4
param_names_planning = ['theta_full_energy', 'theta_low_energy_LC','theta_low_energy_HC', 'theta_basic','beta_dv']

idx = (dat['timeout'] == 0)
response = (dat.loc[idx,['response']] == 0).to_numpy(dtype='int').squeeze()
is_full_energy = dat.loc[idx,['is_full_energy']].to_numpy(dtype='int').squeeze()
is_low_energy_LC = dat.loc[idx,['is_low_energy_LC']].to_numpy(dtype='int').squeeze()
is_low_energy_HC = dat.loc[idx,['is_low_energy_HC']].to_numpy(dtype='int').squeeze()
is_basic = dat.loc[idx,['is_basic']].to_numpy(dtype='int').squeeze()
dv = dat.loc[idx,['dv_planning']].to_numpy().squeeze()
vpn = dat.loc[idx,['vpn']].to_numpy().squeeze() - 100
N_subjects = len(np.unique(vpn))

dat_dict_planning = {'N':len(response),         
            'response':response,
            'dv':dv, 
            'is_full_energy':is_full_energy ,
            'is_low_energy_LC':is_low_energy_LC,
            'is_low_energy_HC':is_low_energy_HC,
            'is_basic':is_basic,   
            'N_subjects':N_subjects,
            'vpn':vpn
            } 

### Hybrid

In [9]:
n_iter = 2000
n_warmup = 1000
n_chains = 4
param_names_hybrid = ['theta_full_energy', 'theta_low_energy_LC','theta_low_energy_HC', 'theta_basic_1','theta_basic_2','theta_basic_3','theta_basic_4','beta_dv']
control_dict = dict(adapt_delta=0.95)

idx = (dat['timeout'] == 0)
response = (dat.loc[idx,['response']] == 0).to_numpy(dtype='int').squeeze()
is_full_energy = dat.loc[idx,['is_full_energy']].to_numpy(dtype='int').squeeze()
is_low_energy_LC = dat.loc[idx,['is_low_energy_LC']].to_numpy(dtype='int').squeeze()
is_low_energy_HC = dat.loc[idx,['is_low_energy_HC']].to_numpy(dtype='int').squeeze()
is_basic_1 = dat.loc[idx,['is_basic_1']].to_numpy(dtype='int').squeeze()
is_basic_2 = dat.loc[idx,['is_basic_2']].to_numpy(dtype='int').squeeze()
is_basic_3 = dat.loc[idx,['is_basic_3']].to_numpy(dtype='int').squeeze()
is_basic_4 = dat.loc[idx,['is_basic_4']].to_numpy(dtype='int').squeeze()
is_basic = dat.loc[idx,['is_basic']].to_numpy(dtype='int').squeeze()
dv = dat.loc[idx,['dv_planning']].to_numpy().squeeze()
vpn = dat.loc[idx,['vpn']].to_numpy().squeeze() - 100
N_subjects = len(np.unique(vpn))

dat_dict_hybrid= {'N':len(response),         
            'response':response,
            'dv':dv,      
            'is_full_energy':is_full_energy ,
            'is_low_energy_LC':is_low_energy_LC,
            'is_low_energy_HC':is_low_energy_HC,
            'is_basic_1':is_basic_1,
            'is_basic_2':is_basic_2,
            'is_basic_3':is_basic_3,
            'is_basic_4':is_basic_4,
            'is_basic':is_basic,
            'N_subjects':N_subjects,
            'vpn':vpn
            } 

## Sampling posterior

In [10]:
res_simple = sm_standard_m1.sampling(data=dat_dict_simple, iter=n_iter,  warmup=n_warmup, thin=1, chains=n_chains,seed=101, verbose = False);

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [11]:
res_planning = sm_standard_m1.sampling(data=dat_dict_planning, iter=n_iter,  warmup=n_warmup, thin=1, chains=n_chains,seed=101, verbose = False);

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [12]:
res_hybrid = sm_standard_m2.sampling(data=dat_dict_hybrid, iter=n_iter,  warmup=n_warmup, thin=1, chains=n_chains,control = control_dict,seed=101, verbose = False);

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


## Computing leave-one-out cross-validation information criterion (LOOIC) for model comparison 

In [13]:
# Convert Stan fit-objects to Arviz inference data
idata_simple = az.from_pystan(posterior=res_simple,log_likelihood='log_lik');
idata_planning = az.from_pystan(posterior=res_planning,log_likelihood='log_lik');
idata_hybrid = az.from_pystan(posterior=res_hybrid,log_likelihood='log_lik');

In [17]:
# Compute LOOIC
looic_simple = az.loo(idata_simple,pointwise=True,scale='deviance')
looic_planning = az.loo(idata_planning,pointwise=True,scale='deviance')
looic_hybrid = az.loo(idata_hybrid,pointwise=True,scale='deviance')